In [1]:
import pandas as pd
from sqlalchemy import create_engine, text, types, String, Integer, Numeric, Float, DATETIME, Text
import config
%load_ext autotime

In [3]:
#remote_eng = create_engine(config.AWS_SQLALCHEMY)

time: 999 µs


In [12]:
# remote_eng = create_engine(config.GCP_SQLALCHEMY)

time: 999 µs


In [19]:
remote_eng = create_engine(config.AZURE_SQLALCHEMY)

time: 980 µs


In [31]:
data_path = 'brazilian-ecommerce/'

time: 1 ms


In [32]:
customers = pd.read_csv(data_path + 'olist_customers_dataset.csv', usecols=['customer_id', 'customer_unique_id',
                                                                                 'customer_zip_code_prefix'])

time: 163 ms


In [22]:
customers.shape

(99441, 3)

time: 12 ms


In [23]:
customer_types = {"customer_id": String(length=32),
        "customer_unique_id": String(length=32),
        "customer_zip_code_prefix": Integer()}

time: 1 ms


In [26]:
customers.to_sql('customers', remote_eng, dtype=customer_types, if_exists='replace', index=False)

time: 27.5 s


In [25]:
remote_eng.execute("ALTER TABLE customers ADD PRIMARY KEY(customer_id)")
remote_eng.execute("CREATE INDEX zip_idx ON customers(customer_zip_code_prefix)")

time: 2.72 s


In [33]:
geolocation = pd.read_csv(data_path + 'olist_geolocation_dataset.csv')
geolocation.shape

(1000163, 5)

time: 601 ms


In [11]:
geo_types = {"geolocation_zip_code_prefix": Integer(),
            "geolocation_lat": Float(),
            "geolocation_lng": Float(),
            "geolocation_city": String(200),
            "geolocation_state": String(10)}

time: 1e+03 µs


In [12]:
geolocation.to_sql('geolocation', remote_eng, dtype=geo_types, if_exists='replace', index=False)

time: 4min


In [13]:
remote_eng.execute("ALTER TABLE geolocation ADD INDEX(geolocation_zip_code_prefix)")

time: 4.65 s


In [34]:
sellers = pd.read_csv(data_path + 'olist_sellers_dataset.csv', usecols=['seller_id', 'seller_zip_code_prefix'])
sellers.shape

(3095, 2)

time: 9 ms


In [15]:
seller_types = {"seller_id": String(32),
               "seller_zip_code_prefix": Integer()}

time: 1.01 ms


In [16]:
sellers.to_sql('sellers', remote_eng, dtype=seller_types, if_exists='replace', index=False)

time: 1.77 s


In [17]:
remote_eng.execute("ALTER TABLE sellers ADD PRIMARY KEY(seller_id)")
remote_eng.execute("ALTER TABLE sellers ADD INDEX(seller_zip_code_prefix)")

time: 1.33 s


In [35]:
categories = pd.read_csv(data_path + 'product_category_name_translation.csv')
categories.shape

(71, 2)

time: 5 ms


In [19]:
category_types = {"product_category_name": String(200),
                 "product_category_name_english": String(200)}

time: 1 ms


In [20]:
categories.to_sql('categories', remote_eng, dtype=category_types, if_exists='replace', index=False)

time: 1.26 s


In [21]:
remote_eng.execute("ALTER TABLE categories ADD PRIMARY KEY(product_category_name)")

time: 855 ms


In [22]:
orders = pd.read_csv(data_path + 'olist_orders_dataset.csv')
orders.shape

(99441, 8)

time: 433 ms


In [34]:
order_types = {"order_id": String(32),
              "customer_id": String(32),
              "order_status": String(100),
              "order_purchase_timestamp": DATETIME,
               "order_approved_at": DATETIME,
               "order_delivered_carrier_date": DATETIME,
               "order_delivered_customer_date": DATETIME,
               "order_estimated_delivery_date": DATETIME
              }

time: 999 µs


In [24]:
orders.to_sql('orders', remote_eng, dtype=order_types, if_exists='replace', index=False)

time: 1min 14s


In [25]:
remote_eng.execute("ALTER TABLE orders ADD PRIMARY KEY(order_id)")
remote_eng.execute("ALTER TABLE orders ADD INDEX(customer_id)")

time: 5.41 s


In [26]:
products = pd.read_csv(data_path + 'olist_products_dataset.csv')
products.shape

(32951, 9)

time: 91 ms


In [27]:
product_types = {"product_id": String(32),
                "product_category_name": String(200),
                "product_name_lenght": Float(),
                "product_description_lenght": Float(),
                "product_photos_qty": Float(),
                "product_weight_g": Float(),
                "product_length_cm": Float(),
                "product_height_cm": Float(),
                "product_width_cm": Float()}

time: 1 ms


In [28]:
products.to_sql('products', remote_eng, dtype=product_types, if_exists='replace', index=False)

time: 11 s


In [29]:
remote_eng.execute("ALTER TABLE products ADD PRIMARY KEY(product_id)")
remote_eng.execute("ALTER TABLE products ADD INDEX(product_category_name)")

time: 2.19 s


In [30]:
reviews = pd.read_csv(data_path + 'olist_order_reviews_dataset.csv')
reviews.shape

(100000, 7)

time: 304 ms


In [31]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
review_id                  100000 non-null object
order_id                   100000 non-null object
review_score               100000 non-null int64
review_comment_title       11715 non-null object
review_comment_message     41753 non-null object
review_creation_date       100000 non-null object
review_answer_timestamp    100000 non-null object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB
time: 29 ms


In [36]:
review_types = {"review_id": String(32),
               "order_id": String(32),
               "review_score": Integer(),
               "review_comment_title": String(200),
                "review_comment_message": Text,
               "review_creation_date": DATETIME,
               "review_answer_timestamp": DATETIME}

time: 3 ms


In [38]:
reviews.to_sql('reviews', remote_eng, dtype=review_types, if_exists='replace', index=False)

time: 57.7 s


In [39]:
remote_eng.execute("ALTER TABLE reviews ADD INDEX(review_id)")
remote_eng.execute("ALTER TABLE reviews ADD INDEX(order_id)")

time: 2.05 s


In [40]:
items = pd.read_csv(data_path + 'olist_order_items_dataset.csv')
items.shape

(112650, 7)

time: 263 ms


In [41]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
order_id               112650 non-null object
order_item_id          112650 non-null int64
product_id             112650 non-null object
seller_id              112650 non-null object
shipping_limit_date    112650 non-null object
price                  112650 non-null float64
freight_value          112650 non-null float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB
time: 23 ms


In [42]:
item_types = {"order_id": String(32),
             "order_item_id": String(32),
             "product_id": String(32),
             "seller_id": String(32),
             "shipping_limit_date": DATETIME,
             "price": Float(),
             "freight_value": Float()}

time: 1e+03 µs


In [43]:
items.to_sql('order_items', remote_eng, dtype=item_types, if_exists='replace', index=False)

time: 55.5 s


In [44]:
remote_eng.execute("ALTER TABLE order_items ADD PRIMARY KEY(order_id, order_item_id)")

time: 4.56 s


In [45]:
remote_eng.execute("ALTER TABLE order_items ADD INDEX(product_id)")

time: 2.04 s


In [46]:
remote_eng.execute("ALTER TABLE order_items ADD INDEX(seller_id)")

time: 1.95 s


In [47]:
payments = pd.read_csv(data_path + 'olist_order_payments_dataset.csv')
payments.shape

(103886, 5)

time: 131 ms


In [48]:
len(payments.order_id.unique())

99440

time: 30 ms


In [52]:
payments_types = {"order_id": String(32),
                 "payment_sequential": Integer(),
                 "payment_type": String(200),
                 "payment_installments": Integer(),
                 "payment_value": Float()}

time: 2 ms


In [50]:
payments.to_sql('payments', remote_eng, dtype=payments_types, if_exists='replace', index=False)

time: 25.4 s


In [51]:
remote_eng.execute("ALTER TABLE payments ADD PRIMARY KEY(order_id, payment_sequential)")

time: 3.02 s
